In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as st
from statsmodels.stats.proportion import proportions_ztest

%matplotlib inline


In [27]:
df = pd.read_csv(r"C:\Users\mombo\OneDrive\Desktop\IRONHACK - COURSE\SEM 5\PROJECT\df_final_demo.txt")
df

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,836976,6.0,73.0,60.5,U,2.0,45105.30,6.0,9.0
1,2304905,7.0,94.0,58.0,U,2.0,110860.30,6.0,9.0
2,1439522,5.0,64.0,32.0,U,2.0,52467.79,6.0,9.0
3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0
4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0
...,...,...,...,...,...,...,...,...,...
70604,7993686,4.0,56.0,38.5,U,3.0,1411062.68,5.0,5.0
70605,8981690,12.0,148.0,31.0,M,2.0,101867.07,6.0,6.0
70606,333913,16.0,198.0,61.5,F,2.0,40745.00,3.0,3.0
70607,1573142,21.0,255.0,68.0,M,3.0,475114.69,4.0,4.0


In [28]:
df1 = pd.read_csv(r"C:\Users\mombo\OneDrive\Desktop\IRONHACK - COURSE\SEM 5\PROJECT\df_final_experiment_clients.txt")

df1

,client_id,Variation
0,9988021,Test
1,8320017,Test
2,4033851,Control
3,1982004,Test
4,9294070,Control
...,...,...
70604,2443347,NaN
70605,8788427,NaN
70606,266828,NaN
70607,1266421,NaN


In [29]:
# merge final_demo + final_experiment_clients
dem_ex=pd.merge(df, df1, on='client_id', how='left')

# drop NaN in column variation 
dem_ex.dropna(subset=['Variation','clnt_tenure_yr', 'clnt_tenure_mnth', 'num_accts', 'calls_6_mnth', 'logons_6_mnth'],inplace= True)

# Handle missing values
print("Missing values per column:\n", dem_ex.isnull().sum())

# column names in lower case
dem_ex.columns = dem_ex.columns.str.lower()

# columns conversions
cols_to_int = ['clnt_tenure_yr', 'clnt_tenure_mnth', 'num_accts', 'calls_6_mnth', 'logons_6_mnth']
dem_ex[cols_to_int] = dem_ex[cols_to_int].astype('Int64')

# drop duplicate from client_id
dem_ex.drop_duplicates(subset=['client_id'], keep='first', inplace=True)

# round column bal 
dem_ex['bal'] = dem_ex['bal'].round(2)

dem_ex

Missing values per column:
 client_id           0
clnt_tenure_yr      0
clnt_tenure_mnth    0
clnt_age            1
gendr               0
num_accts           0
bal                 0
calls_6_mnth        0
logons_6_mnth       0
Variation           0
dtype: int64


,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,variation
0,836976,6,73,60.5,U,2,45105.30,6,9,Test
1,2304905,7,94,58.0,U,2,110860.30,6,9,Control
2,1439522,5,64,32.0,U,2,52467.79,6,9,Test
3,1562045,16,198,49.0,M,2,67454.65,3,6,Test
4,5126305,12,145,33.0,F,2,103671.75,0,3,Control
...,...,...,...,...,...,...,...,...,...,...
50495,1780858,21,262,68.5,M,3,372100.59,6,9,Test
50496,6967120,21,260,68.5,M,3,4279873.38,6,9,Control
50497,5826160,20,249,56.5,F,2,44837.16,2,5,Test
50498,8739285,19,229,69.5,F,2,44994.24,1,4,Test


In [30]:
pt1 = pd.read_csv(r"C:\Users\mombo\OneDrive\Desktop\IRONHACK - COURSE\SEM 5\PROJECT\df_final_web_data_pt_1.txt", sep=',')
pt1

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
...,...,...,...,...,...
343136,2443347,465784886_73090545671,136329900_10529659391_316129,confirm,2017-03-31 15:15:46
343137,2443347,465784886_73090545671,136329900_10529659391_316129,step_3,2017-03-31 15:14:53
343138,2443347,465784886_73090545671,136329900_10529659391_316129,step_2,2017-03-31 15:12:08
343139,2443347,465784886_73090545671,136329900_10529659391_316129,step_1,2017-03-31 15:11:37


In [31]:
pt2 = pd.read_csv(r"C:\Users\mombo\OneDrive\Desktop\IRONHACK - COURSE\SEM 5\PROJECT\df_final_web_data_pt_2.txt", sep=',')
pt2

,client_id,visitor_id,visit_id,process_step,date_time
0,763412,601952081_10457207388,397475557_40440946728_419634,confirm,2017-06-06 08:56:00
1,6019349,442094451_91531546617,154620534_35331068705_522317,confirm,2017-06-01 11:59:27
2,6019349,442094451_91531546617,154620534_35331068705_522317,step_3,2017-06-01 11:58:48
3,6019349,442094451_91531546617,154620534_35331068705_522317,step_2,2017-06-01 11:58:08
4,6019349,442094451_91531546617,154620534_35331068705_522317,step_1,2017-06-01 11:57:58
...,...,...,...,...,...
412259,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:46:10
412260,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:45:29
412261,9668240,388766751_9038881013,922267647_3096648104_968866,step_1,2017-05-24 18:44:51
412262,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:44:34


In [32]:
# Combine the two web data files
web_data = pd.concat([pt1, pt2], axis=0, ignore_index=True)

# Drop duplicates
web_data.drop_duplicates(inplace=True)

# Handle missing values
print("Missing values per column:\n", web_data.isnull().sum())

# Drop rows missing key identifiers
web_data.dropna(subset=['client_id', 'visitor_id', 'visit_id', 'process_step', 'date_time'], inplace=True)

# Convert date column to datetime
web_data['date_time'] = pd.to_datetime(web_data['date_time'], errors='coerce')

# Drop rows where date_time failed conversion
web_data = web_data.dropna(subset=['date_time'])

# Ensure correct data types
web_data['client_id'] = web_data['client_id'].astype(int)

# Sort by client and time
web_data.sort_values(by=['client_id', 'date_time'], inplace=True)

# Reset index
web_data.reset_index(drop=True, inplace=True)

Missing values per column:
 client_id       0
visitor_id      0
visit_id        0
process_step    0
date_time       0
dtype: int64


In [33]:
# Merge demographics + experiment data and web data
merged = web_data.merge(dem_ex, on='client_id', how='left')
merged = merged.dropna()

merged.head()

,client_id,visitor_id,visit_id,process_step,date_time,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,variation
12,555,402506806_56087378777,637149525_38041617439_716659,start,2017-04-15 12:57:56,3,46,29.5,U,2,25454.66,2,6,Test
13,555,402506806_56087378777,637149525_38041617439_716659,step_1,2017-04-15 12:58:03,3,46,29.5,U,2,25454.66,2,6,Test
14,555,402506806_56087378777,637149525_38041617439_716659,step_2,2017-04-15 12:58:35,3,46,29.5,U,2,25454.66,2,6,Test
15,555,402506806_56087378777,637149525_38041617439_716659,step_3,2017-04-15 13:00:14,3,46,29.5,U,2,25454.66,2,6,Test
16,555,402506806_56087378777,637149525_38041617439_716659,confirm,2017-04-15 13:00:34,3,46,29.5,U,2,25454.66,2,6,Test


In [34]:
merged.shape

(317123, 14)

COMPLETION RATE

COMPLETION RATE BY STEPS

In [35]:
# counting of unique visit per stage and per group

step_counts_multiindex = merged.groupby(['process_step', 'variation'])['visit_id'].nunique().sort_values(ascending=False)

step_counts = step_counts_multiindex.unstack(fill_value=0)


#Determining the number of visits at 'start' (base 100%)
start_counts = step_counts.loc['start']
start_control = start_counts.get('Control', 1)
start_test = start_counts.get('Test', 1)

# Calculating rates for each group
completion_rate = pd.DataFrame()
completion_rate['Control'] = (step_counts['Control'] / start_control) * 100
completion_rate['Test'] = (step_counts['Test'] / start_test) * 100

# display order
step_order_ = ['start', 'step_1', 'step_2', 'step_3', 'confirm']

# Reindexing the DataFrame to enforce the order
completion_rate_by_steps = completion_rate.reindex(step_order_, fill_value=0).round(2)


completion_rate_by_steps

,Control,Test
process_step,,
start,100.00,100.00
step_1,76.18,85.31
step_2,65.14,73.90
step_3,59.19,66.92
confirm,51.90,65.55


TIME SPENT ON EACH STEP 

ERROR RATE 

In [36]:
merged['date_time'] = pd.to_datetime(merged['date_time'])

# define the numerical order of the process steps
step_order_merged = {
    'start': 1,
    'step_1': 2,
    'step_2': 3,
    'confirm': 4
}
# New column with step order
merged['step_order_merged'] = merged['process_step'].map(step_order_merged)
# Sort the data by session and by time
merged= merged.sort_values(by=['visitor_id', 'visit_id', 'date_time']).reset_index(drop=True)

#Identify the order of the next step for each line in the session
merged['next_step_merged_order'] = merged.groupby(['visitor_id', 'visit_id'])['step_order_merged'].shift(-1)
#Mark 'Backward' Steps (Errors) / An error occurs when: Current Step Order > Next Step Order
merged['error'] = np.where(merged['step_order_merged'] > merged['next_step_merged_order'], 1, 0)

merged.head()



,client_id,visitor_id,visit_id,process_step,date_time,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,variation,step_order_merged,next_step_merged_order,error
0,6265388,100019786_4272121951,703380671_89376183829_330646,start,2017-04-15 15:13:33,7,94,42.5,U,4,214896.6,5,8,Test,1.0,2.0,0
1,6265388,100019786_4272121951,703380671_89376183829_330646,step_1,2017-04-15 15:13:46,7,94,42.5,U,4,214896.6,5,8,Test,2.0,3.0,0
2,6265388,100019786_4272121951,703380671_89376183829_330646,step_2,2017-04-15 15:14:21,7,94,42.5,U,4,214896.6,5,8,Test,3.0,NaN,0
3,6265388,100019786_4272121951,703380671_89376183829_330646,step_3,2017-04-15 15:15:50,7,94,42.5,U,4,214896.6,5,8,Test,NaN,4.0,0
4,6265388,100019786_4272121951,703380671_89376183829_330646,confirm,2017-04-15 15:16:18,7,94,42.5,U,4,214896.6,5,8,Test,4.0,4.0,0


In [37]:

#Calculate the total errors and the total steps taken per 'variation' group
error_rate_kpi = merged.groupby('variation').agg(
    total_backward_steps=('error', 'sum'),
    total_steps_taken=('client_id', 'count') )

error_rate_kpi

,total_backward_steps,total_steps_taken
variation,,
Control,5331,140482
Test,11589,176641


In [38]:
# Calculer le Taux d'Erreur (%)
error_rate_kpi['Error_Rate'] = (error_rate_kpi['total_backward_steps'] / error_rate_kpi['total_steps_taken']) * 100

error_rate_kpi

,total_backward_steps,total_steps_taken,Error_Rate
variation,,,
Control,5331,140482,3.794792
Test,11589,176641,6.560764


In [ ]:
# We can explain why the error rate in the test group is higher than that in the control group
#  by the fact that the test group is discovering Vanguard's new website.

HYPOTHESIS TESTING  

 test whether the average age of clients engaging with the new process is the same as those engaging with the old process

In [39]:


# Set the hypothesis

#H0 : mu client age_Test = mu client age_Control  
#H1:  mu client age_Test != mu client age_Control                

# significance level= 0.05  

In [40]:
# Extract age samples for Test and Control groups
sample_test_age = merged[merged['variation'] == 'Test']['clnt_age']
sample_control_age = merged[merged['variation'] == 'Control']['clnt_age']


In [41]:
# Perform independent t-test
st.ttest_ind(sample_test_age,sample_control_age)

TtestResult(statistic=np.float64(7.831721878378589), pvalue=np.float64(4.827086764052195e-15), df=np.float64(317121.0))

In [42]:
# P-value is significantly below to the significance level
# we do reject the null hypothesis


# This means that there is a statistically significant difference in average age between 
# customers in the 'Test' and 'Control' processes.

 two-proportion z-test about completion rate

In [43]:
# Extracting step counts  for 'start' and 'confirm'
step_counts_multiindex = merged.groupby(['process_step', 'variation'])['visit_id'].nunique().sort_values(ascending=False)

step_counts = step_counts_multiindex.unstack(fill_value=0)

s_start = step_counts.loc[['start']]
s_confirm = step_counts.loc[['confirm']]

print(s_start)
print("------------------------------")
print(s_confirm)

variation     Control   Test
process_step                
start           30903  33144
------------------------------
variation     Control   Test
process_step                
confirm         16039  21725


In [44]:
#Set the hypothesis

# H0: completion_rate_test = completion_rate_control
# H1: completion_rate_test != completion_rate_control

# significance_level = 0.05

In [45]:
# Perform two-proportion z-test
num_confirm = np.array([21725, 16839])
num_obs = np.array([33144, 30903])
significance_level = 0.05

z_stat_CR = proportions_ztest(count=num_confirm, nobs=num_obs,value=0, alternative='two-sided')


z_stat_CR
 

(np.float64(28.568732923742665), np.float64(1.644038135520575e-179))

In [46]:
#The p-value is less than 0.05: Rejection of the Null Hypothesis (H0)
#The observed difference in completion rate is statistically significant.


#The difference between the completion rate of group Test and the group Control  is not due to chance
#  and is considered statistically significant. The new protocol has a real and demonstrable impact on the completion rate.


one-sided two-proportion z-test --> if completion rate of the Test Group is > 5% than that of Control Group this new version is profitable 

In [47]:
#Set the hypothesis

# H0: completion_rate_test - completion_rate_control <= 0.05
# H1: completion_rate_test - completion_rate_control > 0.05

# significance_level = 0.05

In [48]:
# Extracting step counts  for 'start' and 'confirm'
#  Perform one-tailed two-proportion z-test

num_confirm = np.array([21725, 16839])
num_obs = np.array([33144, 30903])
significance_level = 0.05
threshold_difference=0.05

z_stat_CR_diff = proportions_ztest(count=num_confirm, nobs=num_obs, value=threshold_difference, alternative='larger')
z_stat_CR_diff

(np.float64(15.650418012277488), np.float64(1.6502224957516827e-55))

In [49]:
#The p-value is less than 0.05: we do Reject the Null Hypothesis (H0), in favor of the alternative hypothesis (H1).
#The result is very strong: The completion rate of the Test group is statistically and significantly higher than the completion rate 
# of the Control group increased by more than 5%.
#the new protocol is better, but also cost-effective according to your cost-effectiveness threshold.
    